<a href="https://colab.research.google.com/github/mimitung/testing/blob/main/f_man/strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import sys
# f_man_loc = '../../forecast'
# sys.path.append(f_man_loc)


import pandas as pd
import pickle
from tqdm import tqdm
from IPython.display import display


from   f_man import feature

from f_man import backtest_mtx

from f_man import train

from f_man import tool

def strtran(timeformat):
    if timeformat <10:
        Xtime = '0'+str(timeformat)
    else:
        Xtime = str(timeformat)
    return  Xtime

import re
def formatNum(num):
    num=str(num)
    pattern=r'(\d+)(\d{3})((,\d{3})*)'
    while True:
        num,count=re.subn(pattern,r'\1,\2\3',num)
        if count==0:
            break
    return num

#  <font color=#FF0000>　**gen_first_version**</font>

In [ ]:
def  gen_first_version(direct,estimators,learningrate,predition_list,loop_start,loop_end):
    print('direct:',direct)
    print('estimators:',estimators)
    print('learningrate:',learningrate)
    print('predition_list:',predition_list)
    
    
    import datetime
    import pandas as pd
    daily_reports=pd.DataFrame()
    
    Pstart =datetime.datetime.now()
    print(str(Pstart.year)+'-'+str(Pstart.month)+'-'+str(Pstart.day)+'-'+str(Pstart.hour)+':'+str(Pstart.minute)+':'+str(Pstart.second))
    today=pd.to_datetime(str(Pstart.year)+'-'+str(Pstart.month)+'-'+str(Pstart.day))

    daily_filename =  str(Pstart.year)+\
                        strtran(Pstart.month)+\
                        strtran(Pstart.day)+'_'+\
                        strtran(Pstart.hour)+strtran(Pstart.minute)+'_'\
                        +direct\
                        +'n'+str(estimators)\
                        +'r'+str(learningrate)\
                        +'.pkl'



    
    feature_list = feature.get_feature_list()
    f_ctx = feature.feature_context_init()
    f_ctx = feature.feature_cxt_update(f_ctx,'direct',direct)
    f_ctx = feature.feature_cxt_update(f_ctx,'init_return')
    f_ctx = feature.feature_cxt_update(f_ctx,'backtest')
    
    query_num=len(feature_list)
    for i in tqdm(range(0,query_num,1)):
        for delta_day in range(loop_start,loop_end,1):
            for  estimator in  estimators:

                train_end_day=today+datetime.timedelta(days=-(310+delta_day))
                test_start_day=today+datetime.timedelta(days=-(309+delta_day))
                test_end_day=today+datetime.timedelta(days=-3)

                f_ctx = feature.feature_cxt_update(f_ctx,'name',feature_list[i])

                f_ctx = feature.feature_cxt_update(f_ctx,'traning_features')

                f_ctx = feature.feature_cxt_update(f_ctx,'train_end_day',train_end_day)

                f_ctx = feature.feature_cxt_update(f_ctx,'test_start_day',test_start_day)

                f_ctx = feature.feature_cxt_update(f_ctx,'test_end_day',test_end_day)

                f_ctx = feature.feature_cxt_update(f_ctx,'n_estimators',estimator)

                f_ctx = feature.feature_cxt_update(f_ctx,'learning_rate',learningrate)
                #step2: traing         
                behavior = train.train2(f_ctx,0)


                for min_prediction in predition_list:
                    daily_report =backtest_mtx.back_test2(f_ctx,behavior,min_prediction)
                    daily_reports = pd.concat([daily_reports,daily_report],axis=0).sort_index(ascending=True)            

        daily_reports=daily_reports.reset_index(drop=True)


        pickle.dump(daily_reports, open(daily_filename ,'wb'))



    
    return daily_filename,daily_reports

#  <font color=#FF0000>　**to_gen_best**</font>

In [ ]:
def gen_best(file,win_rate=0.9,win_times=15,reward=10000):
    
    best_choice_filename='best_bull.pkl'
    # PX='build_bull.pkl'
    # 20220210_1626_bullish.pkl
    PX=file

    f = open(PX)
    update_strategy= pickle.load(open(PX, "rb"))
    f.close()

    m1 = update_strategy.win_rate > win_rate
    m2 = update_strategy.win_times >=win_times
    m3 = update_strategy.reward > reward
    m4 = True#update_strategy.learning_rate== 0.05
    m5= True#update_strategy.n_estimators == 300
    m6 =update_strategy.pre_hold >= 0.8

    PA=update_strategy[m1&m2&m3&m4&m5&m6].reset_index(drop=True)
    print(len(PA))
    PA = PA.drop_duplicates(subset=['name','win_rate','win_times','lose_times','prediction_day'],keep='first').reset_index(drop=True)
    print(len(PA))
    #PA=PA.head(5)
    PA=PA
#     pickle.dump(PA, open(best_choice_filename ,'wb'))    
#     display(PA)
    return  PA

#  <font color=#FF0000>　** function: to_gen_result_base_best_stretagy**</font>

In [ ]:
import datetime
import pickle
from tqdm import tqdm
from IPython.display import display

    
def gen_result_base_best_stretagy(best_strategy):

    combin_list=[]
    for  i in range(0,len(best_strategy.index),1):
         combin_list.append([
                             best_strategy.iloc[i]['name'], 
                             best_strategy.iloc[i]['train_eday'], \
                             best_strategy.iloc[i]['test_sday'],\
                             best_strategy.iloc[i]['test_eday'],\
                             best_strategy.iloc[i]['pre_hold'],\
                             best_strategy.iloc[i]['learning_rate'],\
                             best_strategy.iloc[i]['n_estimators']

                            ])
    combin_list       

    direct = best_strategy.iloc[0].direct
    f_ctx = feature.feature_context_init()
    f_ctx = feature.feature_cxt_update(f_ctx,'direct',direct)
    f_ctx = feature.feature_cxt_update(f_ctx,'init_return')
    f_ctx = feature.feature_cxt_update(f_ctx,'backtest')


    
    daily_reports=pd.DataFrame()
    reports=pd.DataFrame()

    # for i in tqdm(range(int(9e6))):
    for i in (range(0,len(combin_list),1)):

            name=combin_list[i][0]
            train_end_day=combin_list[i][1]
            test_start_day=combin_list[i][2]
            test_end_day=combin_list[i][3]
            min_prediction=combin_list[i][4]
            learningrate= combin_list[i][5]
            estimators = combin_list[i][6]

            f_ctx = feature.feature_cxt_update(f_ctx,'name',name)
    #         f_ctx = feature.feature_cxt_update(f_ctx,'traning_features_for_daily')
            f_ctx = feature.feature_cxt_update(f_ctx,'traning_features')
            f_ctx = feature.feature_cxt_update(f_ctx,'prediction_hold',min_prediction)       
            f_ctx = feature.feature_cxt_update(f_ctx,'train_end_day',train_end_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'test_start_day',test_start_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'test_end_day',test_end_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'n_estimators',estimators)
            f_ctx = feature.feature_cxt_update(f_ctx,'learning_rate',learningrate)

    #         display('min_prediction:',min_prediction)   
    #         display('name:',name)
    #         display('learningrate:',learningrate)    

    #         display('train_end_day:',train_end_day)
    #         display('test_start_day:',test_start_day) 

            behavior = train.train2(f_ctx,0)
    #         display(behavior.head(2))

            daily_report,detail_prediction=backtest_mtx.back_test_singal(f_ctx,behavior,min_prediction)
            detail_prediction['name']=name
    #         display(detail_prediction)
    #         print('min_prediction:',min_prediction)
            m1 = detail_prediction.prediction > min_prediction
            detail_prediction = detail_prediction[m1]

    #         display(detail_prediction)

            cond1 =detail_prediction.Profit_rate >1
            cond2= detail_prediction.Profit_rate <= 1

            win_times = len(detail_prediction[cond1])
            lose_times = len(detail_prediction[cond2])

            total =len(detail_prediction)
            if total==0:
                total=1


    #         print ('獲利',win_times,'次/ 成功率{:2.1%}'.format(win_times/total))
    #         print ('虧損',lose_times,'次/ 失敗率{:2.1%}'.format(lose_times/total))
    #         print ('交易次數:',len(detail_prediction))


            reports = pd.concat([reports,detail_prediction],axis=0).sort_index(ascending=True)
    #         break

    # reports
    reports = reports[~reports.index.duplicated(keep='first')]



    cond1 =reports.Profit_rate >1
    cond2= reports.Profit_rate <= 1

    win_times = len(reports[cond1])
    lose_times = len(reports[cond2])
    total =len(reports)
    if  total ==0:
        total=1

    # print('learningrate_check:',learningrate_check)
    print ('獲利',win_times,'次/ 成功率{:2.1%}'.format(win_times/total))
    print ('虧損',lose_times,'次/ 失敗率{:2.1%}'.format(lose_times/total))
    print ('累計獲利:',tool.formatNum(round(reports.gain.sum(),0)*50))
    print ('單次最大虧損:',round(min(reports.gain)*50,0))
    print ('單次最大獲利:',round(max(reports.gain)*50,0))
    print ('交易次數:',len(reports))

    # display(reports)
    reports
    return reports

#  <font color=#FF0000>　**to_get_today_report**</font>

In [ ]:
###### import datetime
import pickle
from tqdm import tqdm
from IPython.display import display

    
def get_today_report(best_strategy):

    combin_list=[]
    for  i in range(0,len(best_strategy.index),1):
         combin_list.append([
                             best_strategy.iloc[i]['name'], 
                             best_strategy.iloc[i]['train_eday'], \
                             best_strategy.iloc[i]['test_sday'],\
                             best_strategy.iloc[i]['test_eday'],\
                             best_strategy.iloc[i]['pre_hold'],\
                             best_strategy.iloc[i]['learning_rate'],\
                             best_strategy.iloc[i]['n_estimators']

                            ])
    combin_list       

    direct = best_strategy.iloc[0].direct
    f_ctx = feature.feature_context_init()
    f_ctx = feature.feature_cxt_update(f_ctx,'direct',direct)
    f_ctx = feature.feature_cxt_update(f_ctx,'init_return')
    f_ctx = feature.feature_cxt_update(f_ctx,'backtest')


    
    today_reports=pd.DataFrame()
    reports=pd.DataFrame()

    # for i in tqdm(range(int(9e6))):
    for i in tqdm(range(0,len(combin_list),1)):

            name=combin_list[i][0]
            train_end_day=combin_list[i][1]
            test_start_day=combin_list[i][2]
            
            now =datetime.datetime.now()
            today=pd.to_datetime(str(now.year)+'-'+str(now.month)+'-'+str(now.day))
            test_end_day=pd.Timestamp(today)

            min_prediction=combin_list[i][4]
            learningrate= combin_list[i][5]
            estimators = combin_list[i][6]

            f_ctx = feature.feature_cxt_update(f_ctx,'name',name)
    #         f_ctx = feature.feature_cxt_update(f_ctx,'traning_features_for_daily')
            f_ctx = feature.feature_cxt_update(f_ctx,'traning_features')
            f_ctx = feature.feature_cxt_update(f_ctx,'prediction_hold',min_prediction)       
            f_ctx = feature.feature_cxt_update(f_ctx,'train_end_day',train_end_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'test_start_day',test_start_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'test_end_day',test_end_day)
            f_ctx = feature.feature_cxt_update(f_ctx,'n_estimators',estimators)
            f_ctx = feature.feature_cxt_update(f_ctx,'learning_rate',learningrate)



            behavior = train.train2(f_ctx,0)
            
            
#             display(behavior)

            today_report = backtest_mtx.today_report(f_ctx,behavior)
    
         
            today_reports = pd.concat([today_reports,today_report],axis=0).sort_index(ascending=True)
            today_reports = today_reports.reset_index(drop=True)
#             display(today_reports)
    
     

    # display(reports)
    today_reports
    return today_reports